In [48]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
!pip install seaborn
#import all libraries and dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as ticker
import matplotlib.ticker as plticker
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

     |████████████████████████████████| 296kB 23.0MB/s eta 0:00:01
You should consider upgrading via the 'pip install --upgrade pip' command.


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:167: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.

In [7]:
world_cup = pd.read_csv('World Cup 2019 Dataset.csv')
results = pd.read_csv('results.csv')

In [8]:
#narrowing to team patcipating in the world cup
worldcup_teams = ['England', ' South Africa', '', 'West Indies', 
            'Pakistan', 'New Zealand', 'Sri Lanka', 'Afghanistan', 
            'Australia', 'Bangladesh', 'India']

In [9]:
#cleaning date column for exploration
results.loc[results.date == 'Jul 3-4, 2010', 'date'] = '03-07-2010'
results.loc[results.date == 'Aug 19-20, 2010', 'date'] = '19-08-2010'
results.loc[results.date == 'Jul 11-12, 2012', 'date'] = '11-07-2013'
results.loc[results.date == 'Jul 2-3, 2013', 'date'] = '02-07-2013'
results.loc[results.date == 'Jul 7-8, 2013', 'date'] = '03-07-2013'
results.loc[results.date == 'Aug 27-28, 2013', 'date'] = '27-08-2013'
results.loc[results.date == 'Dec 10-11, 2014', 'date'] = '10-12-2014'

In [23]:
df_teams_final  = results[(results['Team_1'].isin(worldcup_teams)) & (results['Team_2'].isin(worldcup_teams))]

In [10]:
#df_teams_1 = results[results['Team_1'].isin(worldcup_teams)]
#df_teams_2 = results[results['Team_2'].isin(worldcup_teams)]
#df_teams = pd.concat((df_teams_1, df_teams_2))
#df_teams.drop_duplicates()

,date,Team_1,Team_2,Winner,Margin,Ground
0,4-Jan-10,Bangladesh,Sri Lanka,Sri Lanka,7 wickets,Dhaka
1,5-Jan-10,India,Sri Lanka,Sri Lanka,5 wickets,Dhaka
2,7-Jan-10,Bangladesh,India,India,6 wickets,Dhaka
3,8-Jan-10,Bangladesh,Sri Lanka,Sri Lanka,9 wickets,Dhaka
4,10-Jan-10,India,Sri Lanka,India,8 wickets,Dhaka
...,...,...,...,...,...,...
933,21-May-17,Ireland,New Zealand,New Zealand,190 runs,Dublin (Malahide)
940,3-Jun-17,South Africa,Sri Lanka,South Africa,96 runs,The Oval
986,15-Oct-17,South Africa,Bangladesh,South Africa,10 wickets,Kimberley
988,18-Oct-17,South Africa,Bangladesh,South Africa,104 runs,Paarl


In [30]:
year = []
for row in df_teams_final['date']:
    year.append(int(row[7:]))
df_teams_final['match_year']= year
df_teams_2011 = df_teams_final
#df_teams_2011 = df_teams_final[df_teams_final.match_year >= 11]

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [31]:
#dropping columns that wll not affect match outcomes
df_teams_2011 = df_teams_2011.drop(['date','Margin', 'Ground','match_year'], axis=1)
df_teams_2011.head()

,Team_1,Team_2,Winner
0,Bangladesh,Sri Lanka,Sri Lanka
1,India,Sri Lanka,Sri Lanka
2,Bangladesh,India,India
3,Bangladesh,Sri Lanka,Sri Lanka
4,India,Sri Lanka,India


In [32]:
#Building the model
#the prediction label: The winning_team column will show "1" Team 1 has won and "2" if the away team has won.

df_teams_2011 = df_teams_2011.reset_index(drop=True)
df_teams_2011.loc[df_teams_2011.Winner == df_teams_2011.Team_1,'winning_team']=1
df_teams_2011.loc[df_teams_2011.Winner == df_teams_2011.Team_2, 'winning_team']=2
df_teams_2011 = df_teams_2011.drop(['winning_team'], axis=1)

df_teams_2011.head()

,Team_1,Team_2,Winner
0,Bangladesh,Sri Lanka,Sri Lanka
1,India,Sri Lanka,Sri Lanka
2,Bangladesh,India,India
3,Bangladesh,Sri Lanka,Sri Lanka
4,India,Sri Lanka,India


In [33]:
#convert team-1 and team-2 from categorical variables to continous inputs 
# Get dummy variables
final = pd.get_dummies(df_teams_2011, prefix=['Team_1', 'Team_2'], columns=['Team_1', 'Team_2'])

# Separate X and y sets
X = final.drop(['Winner'], axis=1)
y = final["Winner"]


# Separate train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=101)

In [35]:
final.head()

,Winner,Team_1_Afghanistan,Team_1_Australia,Team_1_Bangladesh,Team_1_England,Team_1_India,Team_1_New Zealand,Team_1_Pakistan,Team_1_Sri Lanka,Team_1_West Indies,Team_2_Afghanistan,Team_2_Australia,Team_2_Bangladesh,Team_2_England,Team_2_India,Team_2_New Zealand,Team_2_Pakistan,Team_2_Sri Lanka,Team_2_West Indies
0,Sri Lanka,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,Sri Lanka,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0
2,India,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3,Sri Lanka,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,India,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0


In [47]:
rf = RandomForestClassifier(n_estimators=100, max_depth=7,
                              random_state=101)
rf.fit(X_train, y_train) 


score = rf.score(X_train, y_train)
score2 = rf.score(X_test, y_test)


print("Training set accuracy: ", '%.3f'%(score))
print("Test set accuracy: ", '%.3f'%(score2))

Training set accuracy:  0.701
Test set accuracy:  0.633


/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:489: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y_store_unique_indices = np.zeros(y.shape, dtype=np.int)
/opt/conda/lib/python3.7/site-packages/sklearn/tree/tree.py:163: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use,

In [49]:
#leveraging ICC rankings
#the team which is positioned higher on the ICC Ranking will be considered "favourite" for the match
#and therefore, will be positioned under the "Team_1" column

# Loading new datasets
ranking = pd.read_csv('icc_rankings.csv') 
fixtures = pd.read_csv('fixtures.csv')

In [50]:
# Create new columns with ranking position of each team
fixtures.insert(1, 'first_position', fixtures['Team_1'].map(ranking.set_index('Team')['Position']))
fixtures.insert(2, 'second_position', fixtures['Team_2'].map(ranking.set_index('Team')['Position']))

# We only need the group stage games, so we have to slice the dataset
fixtures = fixtures.iloc[:45, :]
fixtures.tail()

,Round Number,first_position,second_position,Date,Location,Team_1,Team_2,Group,Result
40,1,1.0,4.0,3/7/2019,"Riverside Ground, Chester-le-Street",England,New Zealand,Group A,NaN
41,1,10.0,8.0,4/7/2019,"Headingley, Leeds",Afghanistan,West Indies,Group A,NaN
42,1,6.0,7.0,5/7/2019,"Lord's, London",Pakistan,Bangladesh,Group A,NaN
43,1,9.0,2.0,6/7/2019,"Headingley, Leeds",Sri Lanka,India,Group A,NaN
44,1,5.0,3.0,6/7/2019,"Emirates Old Trafford, Manchester",Australia,South Africa,Group A,NaN


In [53]:
# Loop to add teams to new prediction dataset based on the ranking position of each team
pred_set = []
for index, row in fixtures.iterrows():
    if row['first_position'] < row['second_position']:
        pred_set.append({'Team_1': row['Team_1'], 'Team_2': row['Team_2'], 'winning_team': None})
    else:
        pred_set.append({'Team_1': row['Team_2'], 'Team_2': row['Team_1'], 'winning_team': None})
        
pred_set = pd.DataFrame(pred_set)
backup_pred_set = pred_set
pred_set.head()

,Team_1,Team_2,winning_team
0,England,South Africa,None
1,Pakistan,West Indies,None
2,New Zealand,Sri Lanka,None
3,Australia,Afghanistan,None
4,South Africa,Bangladesh,None


In [54]:
# Get dummy variables and drop winning_team column
pred_set = pd.get_dummies(pred_set, prefix=['Team_1', 'Team_2'], columns=['Team_1', 'Team_2'])

# Add missing columns compared to the model's training dataset
missing_cols = set(final.columns) - set(pred_set.columns)
for c in missing_cols:
    pred_set[c] = 0
pred_set = pred_set[final.columns]


pred_set = pred_set.drop(['Winner'], axis=1)
pred_set.head()

,Team_1_Afghanistan,Team_1_Australia,Team_1_Bangladesh,Team_1_England,Team_1_India,Team_1_New Zealand,Team_1_Pakistan,Team_1_Sri Lanka,Team_1_West Indies,Team_2_Afghanistan,Team_2_Australia,Team_2_Bangladesh,Team_2_England,Team_2_India,Team_2_New Zealand,Team_2_Pakistan,Team_2_Sri Lanka,Team_2_West Indies
0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0
3,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0


In [59]:
#group matches 
def group_stage_predictions(pred_set):
    
    predictions = rf.predict(pred_set)
    my_groupstage_predictions = pd.DataFrame(columns=['Team_1','Team_2','Predicted_winner'])
    team_1_list = []
    team_2_list = []
    winner_list = []
    for i in range(fixtures.shape[0]):
        #print(backup_pred_set.iloc[i, 1] + " and " + backup_pred_set.iloc[i, 0])
        if predictions[i] == 1:
            temp =     backup_pred_set.iloc[i, 1]
            #print("Winner: " + backup_pred_set.iloc[i, 1])
    
        else:
            temp =     backup_pred_set.iloc[i, 0]
            #print("Winner: " + backup_pred_set.iloc[i, 0])
        team_1_list.append(backup_pred_set.iloc[i, 1])
        team_2_list.append(backup_pred_set.iloc[i, 0])
        winner_list.append(temp)

    my_groupstage_predictions['Team_1'] = team_1_list
    my_groupstage_predictions['Team_2'] = team_2_list
    my_groupstage_predictions['Predicted_winner'] = winner_list
    
    return my_groupstage_predictions

In [60]:
group_Stage_output = group_stage_predictions(pred_set)

In [68]:
group_Stage_output.groupby(['Predicted_winner'])

ValueError: No axis named <built-in function sum> for object type DataFrame